In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=120, return_messages=True, memory_key="history"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


# RunnablePassthrough을 사용하여 필요한 데이터를 먼저 load하고 그 값을 history라는 이름으로 prompt에 전달한다.
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


invoke_chain("My name is nico")
invoke_chain("What is my name?")

In [ ]:
from langchain.schema import BaseOutputParser


# 받은 문자열을 ,을 기준으로 리스트로 만든다.
class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        # list 각 항목에 다시 str.strip 함수를 적용
        # str.strip 함수는 각 문자열의 앞 뒤 공백을 제거한다.
        return list(map(str.strip, items))


parser = CommaOutputParser()

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everythig you are asked will be answered with a comma seperated list of max {max_items}. Do not reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

# chain은 PromptTemplate | Model | OutputParser 로 만들 수 있다.
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "what are animals?"})

In [ ]:
from langchain.llms.openai import OpenAI  # 기본값으로 davinchi-003를사용
from langchain.chat_models import ChatOpenAI  # 기본값으로 GTP3.5-Turbo를 사용
from langchain.schema import HumanMessage, AIMessage, SystemMessage, BaseOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate

# streaming=True, callbacks=[StreamingStdOutCallbackHandler()] 옵션을 넣어주면 실시간 출력을 볼 수 있다.
chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

# 답번의 예시
examples = [
    {
        "question": "What do you know about Japan?",
        "answer": """
        I know this:
        Capital: Tokyo
        Languae: Japanese
        Food: Sushi
        Currency: Yen
        """,
    }
]

# 답변 양식에 대한 prompt
example_prompt = PromptTemplate.from_template("Human:{question}|nAI:{answer}")

# 질문에 대한 prompt 작성
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    # suffix가 유저의 질문 부분임
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Korea")
# 결과: 'Human:What do you know about Japan?|nAI:\n  I know this:\n  Capital: Tokyo\n  Languae: Japanese\n  Food: Sushi\n   :Currency: Yen\n   \n\nHuman: What do you know about Korea?'

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

# 랭체인을 사용해 얻은 값을 자동으로 메모리에 캐싱하도록 한다.
set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everythig you are asked will be answered with a comma seperated list of max {max_items}. Do not reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

# 다음 chain은 PromptTemplate | Model | OutputParser 형태로 이루어져 있다.
chain = template | chat | CommaOutputParser()
# 결과: ['Mammals', 'Birds', 'Reptiles', 'Amphibians', 'Fish']
chain.invoke({"max_items": 5, "question": "what are animals?"})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

# get_openai_callback()를 사용하면 openAI를 사용한 결과를 받아올 수 있다.(챗 결과랑 다른거임)
with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    # 위 처리에 소요된 비용을 출력
    print(usage)

In [ ]:
from langchain.llms.openai import OpenAI  # 기본값으로 davinchi-003를사용
from langchain.chat_models import ChatOpenAI  # 기본값으로 GTP3.5-Turbo를 사용
from langchain.schema import HumanMessage, AIMessage, SystemMessage, BaseOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

# streaming=True, callbacks=[StreamingStdOutCallbackHandler()] 옵션을 넣어주면 실시간 출력을 볼 수 있다.
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "country": "Japan",
        "answer": """
        I know this:
        Capital: Tokyo
        Languae: Japanese
        Food: Sushi
        :Currency: Yen
        """,
    }
]

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "What do you know about {country}?"), ("ai", "{answer}")]
)


example_prompt = FewShotChatMessagePromptTemplate(
    # ChatPromptTemplate을 사용해서 답변 양식을 만든다.
    example_prompt=example_prompt,
    # 실제 답변의 예시를 만든다.
    examples=examples,
)


final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert"),
        # 이전에 니가 이런식으로 답변했다는 것을 example_prompt로 예를 들어준다.
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Korea"})

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Lee, I live in South Korea", "Wow, that is so cool!")
add_message("South Korea is getting fallen.", "what? why?")

get_history()

In [ ]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm LEE, I live in South Korea", "Wow that is so cool!")
memory.load_memory_variables({"input": "who is LEE"})

In [ ]:
add_message("LEE likes kimchi", "Wow that is so cool!")
memory.load_memory_variables({"inputs": "what does LEE like"})

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = TextLoader("./files/textbook.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


# 해당 함수에서 돌려주는 값은 context에 들어가는 값이기 때문에 하나의 string을 반환해야한다.
# 그렇기 때문에 inputs에 들어 있는 모든 데이터를 하나의
def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]

    return "\n\n".join(
        map_doc_chain.invoke(
            {
                "context": doc.page_content,
                "question": question,
            }
        ).content
        for doc in documents
    )


# documents: vectorstore에서 받은 Documents
# question: vectorstore에서 값을 받기 위한 질문
# RunnableLambda를 사용하면 chain의 어느 위치에서도 함수 실행 가능
# RunnableLambda에서 map_docs 함수를 실행하는데 자동으로 map_docs에 필요한 파라미터로써 앞에서 실시한 "documents": retriever, "question": RunnablePassthrough() 값이 들어간다.
# 참고로 "documents": retriever, "question": RunnablePassthrough()은 사전형 데이터로 inputs 파라미터에 들어간다.
map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

# 요약받은 Doument(=context)에 prompt를 적용시켜 llm을 한다.
chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("Where does Winston live?")

AIMessage(content='Winston lives in Victory Mansions, specifically on the seventh floor of the building.')

In [19]:
from gpt4all import GPT4All
model = GPT4All(model_name='orca-mini-3b-gguf2-q4_0.gguf')
with model.chat_session():
    response1 = model.generate(prompt='hello', temp=0)
    response2 = model.generate(prompt='write me a short poem', temp=0)
    response3 = model.generate(prompt='thank you', temp=0)
    print()

    for message in model.current_chat_session:
        role = message['role']
        content = message['content']
        
        if role == 'system':
            print(f"SYSTEM:  {content}")
        elif role == 'user':
            print(f"USER: {content}")
        else:
            print(f"ASSISTANT: {content}")
            print("-----------")
            print("")
        



SYSTEM:  ### System:
You are an AI assistant that follows instruction extremely well. Help as much as you can.
USER: hello
ASSISTANT:  Hello! How may I assist you today?
-----------

USER: write me a short poem
ASSISTANT:  Sure, here's a short poem for you: 

Beneath the bright sun, 
The flowers bloom and dance. 
The birds sing sweetly, 
As butterflies flutter by. 

In this world of chaos, 
We find peace and calm. 
And in each moment we seize, 
Our lives are filled with light.
-----------

USER: thank you
ASSISTANT:  You're welcome! Is there anything else I can help you with?
-----------

